In [10]:

import numpy as np
import torch
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution3D, ZeroPadding3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

meta_data = './dataset/metadata/participant_d040722.csv'


In [11]:
metadata = pd.read_csv(meta_data)
metadata = metadata.query('rndgroup == 1 ')

#metadata
metadata = metadata[[ 'pid','lung_cancer', 'conflc', 'canc_rpt_link', 'cancyr', 'can_scr', 'last_screen_studyyr' ]]

def get_labels(pids, metadata_df):
    labels = metadata_df[metadata_df['pid'].isin(pids)]
    return pids, labels

cancerous = metadata.query('lung_cancer == 1 and canc_rpt_link == 1 and can_scr == 1 and cancyr == 0')
non_cancerous = metadata.query('lung_cancer == 0 and last_screen_studyyr == 0 ' )

test_positive = cancerous.iloc[170:]
train_positive = cancerous.iloc[0:170]
train_sample  = train_positive.sample(n = 110)

#return comma separated pids
def cs_pid(dataframe):

    comma_seperated_pids = ''
    for i, row in dataframe.iterrows():
        pid = row['pid'].astype(int)
        comma_seperated_pids += f' ,{pid}'
    print(comma_seperated_pids)





C:\Users\Woody\AppData\Local\Temp\ipykernel_10968\2981791683.py:1: DtypeWarning: Columns (239,240,348) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(meta_data)


In [12]:
test_negetive = non_cancerous.iloc[600:]
train_negetive = non_cancerous.iloc[0:800]
train_sample  = train_negetive.sample(n = 200)

#return comma separated pids
cs_pid(train_sample)

 ,202728 ,132489 ,128981 ,107953 ,117687 ,101706 ,119413 ,104647 ,133248 ,207666 ,127615 ,206244 ,122461 ,133989 ,206533 ,202416 ,206352 ,202473 ,121358 ,200862 ,200317 ,132403 ,100877 ,117554 ,107513 ,206847 ,116524 ,207539 ,209812 ,209000 ,205174 ,128174 ,109788 ,117408 ,200474 ,103925 ,117323 ,109740 ,128777 ,123330 ,129829 ,118556 ,111169 ,208124 ,102962 ,112755 ,114271 ,206644 ,108509 ,131536 ,104448 ,113778 ,202050 ,126602 ,202982 ,124188 ,104079 ,129565 ,207093 ,101463 ,118903 ,129407 ,130034 ,210378 ,100541 ,100503 ,120110 ,129179 ,207059 ,130452 ,106961 ,209450 ,122270 ,130824 ,210129 ,132446 ,107059 ,203799 ,124823 ,114174 ,124101 ,104783 ,101932 ,108024 ,208225 ,131451 ,116473 ,102303 ,117858 ,210892 ,112257 ,207484 ,106202 ,203066 ,123469 ,200473 ,126687 ,128214 ,102369 ,200446 ,124937 ,209635 ,111385 ,114063 ,209035 ,129404 ,209528 ,127794 ,201250 ,113058 ,100766 ,127682 ,102802 ,201264 ,204998 ,131748 ,203787 ,101184 ,120551 ,208808 ,103428 ,114054 ,100886 ,115411 ,127779

In [13]:
import os 

def return_scan_path(path, sliceThreshold):
    scans_path = []
    for d in os.listdir(path):
        if os.path.isdir(f'{path}\{d}') :
            for dd in os.listdir(f'{path}\{d}'):
                if os.path.isdir(f'{path}\{d}\{dd}') :
                    for ddd in os.listdir(f'{path}\{d}\{dd}'):
                        if len(os.listdir(f'{path}\{d}\{dd}\{ddd}')) >= sliceThreshold: 
                            scans_path.append(f'{path}\{d}\{dd}\{ddd}')

    return scans_path
return_scan_path(r'c:\Users\Woody\Desktop\lung cancer\dataset\train_positive\manifest-1681052108196\NLST', 100)

['c:\\Users\\Woody\\Desktop\\lung cancer\\dataset\\train_positive\\manifest-1681052108196\\NLST\\100518\\01-02-1999-NA-NLST-LSS-27017\\2.000000-0OPAGELSQXD3702.5120.00000064.00.11.5-08555',
 'c:\\Users\\Woody\\Desktop\\lung cancer\\dataset\\train_positive\\manifest-1681052108196\\NLST\\101287\\01-02-1999-NA-NLST-LSS-69722\\3.000000-0OPATOAQUL4FC51276.62-52642',
 'c:\\Users\\Woody\\Desktop\\lung cancer\\dataset\\train_positive\\manifest-1681052108196\\NLST\\101387\\01-02-1999-NA-NLST-LSS-10854\\2.000000-0OPAGELSQXD3202.514040.00.01.5-63002',
 'c:\\Users\\Woody\\Desktop\\lung cancer\\dataset\\train_positive\\manifest-1681052108196\\NLST\\101397\\01-02-1999-NA-NLST-LSS-87367\\2.000000-0OPAGELS16B3202.512060.00.11.375-93455',
 'c:\\Users\\Woody\\Desktop\\lung cancer\\dataset\\train_positive\\manifest-1681052108196\\NLST\\101834\\01-02-1999-NA-NLST-LSS-26156\\2.000000-0OPAGELSQXD3202.514040.00.01.5-52093',
 'c:\\Users\\Woody\\Desktop\\lung cancer\\dataset\\train_positive\\manifest-168105210

In [19]:
# data loader 
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

batch_size = 10
positives = np.array(os.listdir(f'c:/Users/Woody/Desktop/lung cancer/dataset/train_positive/manifest-1681052108196/segmented/'))
negetives = np.array(os.listdir(f'c:/Users/Woody/Desktop/lung cancer/dataset/train_negetive/manifest-1681053813358/segemented/'))
data = np.concatenate((positives, negetives))


def get_labels_from_fnames(names, metadata_df):
    pids = np.array([int(name[:-3]) for name in names ])
    labels = metadata_df[metadata_df['pid'].isin(pids)]

    reindexed_labels = labels.set_index('pid')
    reindexed_labels = reindexed_labels.reindex(pids)['lung_cancer'].values
    return pids, reindexed_labels

x, y = get_labels_from_fnames(data, metadata)

# test size should be changed 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.9, random_state=42)


def get_labels(pids, metadata_df):
    labels = metadata_df[metadata_df['pid'].isin(pids)]
    return pids, labels



class lungs_dataset(Dataset):
    def __init__(self, x, y , path ):
        self.x = x 
        self.y = y 
        self.path = path
        


    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        pid = self.x[idx]
        label = self.y[idx]

        fname = f'{self.path}{pid}.pt'
        tensor = torch.load(fname)

        return tensor , label

path_to_sdata = f'c:/Users/Woody/Desktop/lung cancer/dataset/segmented_lungs/' 
training_data = lungs_dataset(X_train, y_train, path_to_sdata)
#tloader = training data loader 
tloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
#iter_tloader = iter(tloader)



In [23]:
import torch.nn as nn 
import torch
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 3D convolutional layers
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3)
        
        # Max pooling layer
        self.pool = nn.MaxPool3d(kernel_size=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(in_features=128 * 6 * 6 * 6, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=10)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self, x):
        # Pass input through 3D convolutional layers and max pooling layer
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        # Flatten the output from 3D convolutional layers
        x = x.view(-1, 128 * 6 * 6 * 6)
        
        # Pass flattened output through fully connected layers
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x
    

net = Net()
summary( net , (1 ,320, 410, 410) )
print(net)

In [20]:
epochs = 10
train_losses, test_losses, accuracy  = [] , [] , []
learning_rate = 0.001


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr= learning_rate)



In [ ]:
total = 0 
correct = 0
for epoch in range(epochs):
    running_loss = 0
    for i, data in enumerate(tloader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        
        print(outputs)
        print(labels)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        train_losses.append(loss.item())
        with torch.no_grad():
            patch_correct = np.count_nonzero(np.round(outputs) == labels)
            correct += patch_correct
            total += batch_size
            accuracy.append(correct/ total)
            print(f'patch_correct is : {patch_correct} \n output is {np.round(outputs) } \n labes are { labels} ')
            


        
        if i % 15 == 14:
            print(f'running loss for epoch {epoch } is {running_loss / 15}')
            running_loss = 0
        

In [ ]:

def create_model():


    print("Creating model.")
    # sys.stdout. Flush()
    nClasses=2
    model = Sequential()
    #model.add(ZeroPadding3d((0, 1, 1), input_shape  = (3, 320, 320, 1)))
    model.add(Convolution3D(32, (3, 3, 3), activation = 'relu', input_shape = (3, 320, 320, 1)))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(64, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), padding='valid'))



    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(128, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(128, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), padding='valid'))

    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(256, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(256, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(256, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), padding='valid'))

    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(512, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), padding='valid'))

    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(512, (3, 3, 3), activation= 'relu'))
    model.add(ZeroPadding3D((1, 1, 1)))
    model.add(Convolution3D(512, (3, 3, 3), activation= 'relu'))
    model.add(MaxPooling3D((2, 2, 2), padding='valid'))

    model.add(Flatten())
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nClasses, activation= 'softmax'))
    # print (model. summary())

    print("Done!")

    #sys.stdout.lush()
create_model()





'''data_generator = ImageDataGenerator(
            rotation_range=0,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
)'''
#flow from the directory or flow


Creating model.


ValueError: Exception encountered when calling layer "max_pooling3d_1" (type MaxPooling3D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling3d_1/MaxPool3D}} = MaxPool3D[T=DT_FLOAT, data_format="NDHWC", ksize=[1, 2, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 2, 1]](Placeholder)' with input shapes: [?,1,318,318,64].

Call arguments received by layer "max_pooling3d_1" (type MaxPooling3D):
  • inputs=tf.Tensor(shape=(None, 1, 318, 318, 64), dtype=float32)